# README    
1. generate a Keras Layer :AttLayer  
2. Use csv module to import data


In [1]:
import numpy as np
import pandas as pd
import sys
import os

os.environ['KERAS_BACKEND']='theano'
from keras.engine.topology import Layer
from keras import initializations
from keras import backend as K

from collections  import defaultdict
import pandas as pd
import numpy as np

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5105)


In [2]:
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):


        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        '''
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])


#Attention GRU network
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        '''   
        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

# Read Data

In [3]:
datafile  =  '/home/yinqijin/WorkSpace/2.RNA_Structure_Profile/Orig_data/pairs.csv'
import csv

In [4]:
with open(datafile) as csvfile:
    spamreader = csv.reader(csvfile)
    for  row in spamreader:
        csvkeys = row
        break
print csvkeys

['bin', 'enhancer_chrom', 'enhancer_distance_to_promoter', 'enhancer_end', 'enhancer_name', 'enhancer_start', 'label', 'promoter_chrom', 'promoter_end', 'promoter_name', 'promoter_start', 'window_end', 'window_start', 'window_chrom', 'window_name', 'interactions_in_window', 'active_promoters_in_window']


In [5]:
csvdata = dict()
for item in csvkeys:
    csvdata[item] =[]
with open(datafile) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # Add values of every keys to the csvdata for every row
        for item in csvkeys:
            csvdata[item].append(row[item])
            if abs(int(row['enhancer_start'] )-int( row['enhancer_end'])) <6:
                print row
    

{'bin': '(283831.4, 507695]', 'promoter_name': 'GM12878|chr17:40728821-40732200', 'enhancer_chrom': 'chr17', 'window_end': '41080594', 'active_promoters_in_window': '8', 'window_chrom': 'chr17', 'enhancer_distance_to_promoter': '348393', 'label': '1', 'promoter_end': '40732200', 'promoter_start': '40728821', 'enhancer_end': '41080600', 'window_start': '40732201', 'interactions_in_window': '1', 'enhancer_start': '41080595', 'promoter_chrom': 'chr17', 'window_name': 'GM12878|chr17:40732201-41080594', 'enhancer_name': 'GM12878|chr17:41080595-41080600'}
{'bin': '(283831.4, 507695]', 'promoter_name': 'GM12878|chr17:40728821-40732200', 'enhancer_chrom': 'chr17', 'window_end': '41080594', 'active_promoters_in_window': '8', 'window_chrom': 'chr17', 'enhancer_distance_to_promoter': '348393', 'label': '1', 'promoter_end': '40732200', 'promoter_start': '40728821', 'enhancer_end': '41080600', 'window_start': '40732201', 'interactions_in_window': '1', 'enhancer_start': '41080595', 'promoter_chrom':

In [6]:
#print csvdata.keys()
for i in range(1):
    print '-'*40
    for item in csvkeys:
        print item ,': ',csvdata[item][5000]

----------------------------------------
bin :  [22271, 83570.8]
enhancer_chrom :  chr22
enhancer_distance_to_promoter :  32514
enhancer_end :  43515000
enhancer_name :  GM12878|chr22:43514400-43515000
enhancer_start :  43514400
label :  0
promoter_chrom :  chr22
promoter_end :  43548957
promoter_name :  GM12878|chr22:43547516-43548957
promoter_start :  43547516
window_end :  43547515
window_start :  43515001
window_chrom :  chr22
window_name :  GM12878|chr22:43515001-43547515
interactions_in_window :  0
active_promoters_in_window :  1


## Make Promoter/Enhancer Sequence

In [7]:
from pyfasta import Fasta
genome = Fasta('/home/yinqijin/WorkSpace/DataHub/genome.fa')

In [9]:
#证明序列不会有错
import numpy as np
chrom = np.unique(csvdata['enhancer_chrom'])
print chrom
for item in chrom:
    #print genome[item][100:102]

IndentationError: expected an indented block (<ipython-input-9-66f86077914a>, line 6)

In [10]:
#获得ｋｍｅｒ的函数
d2 = {'a':0, 'A':0, 'g':1, 'G':1, 'c':2, 'C':2, 't':3, 'T':3, 'N':0, 'n':0}
def seq_to_kspec(seq):
    mat = np.zeros((4096,1))
    k = 0
    if len(seq)<6:
        return mat
    for i in range(6):
        k = k*4 + d2[seq[i]]
    mat[k]+=1  
    for i in range(6,len(seq)):
        k = k - 4**5*d2[seq[i-6]]
        k = k*4 + d2[seq[i]]
        mat[k] += 1  
    return mat

In [11]:
#获得４＊Ｌ维０１矩阵的函数
d = {'a':0, 'A':0, 'g':1, 'G':1, 'c':2, 'C':2, 't':3, 'T':3, 'N':4, 'n':4}
def seq_to_mat(seq):
    mat = np.zeros((len(seq),5))  #把N量生成新的一行 然后舍去
    for i in range(len(seq)):
        mat[i,d[seq[i]]] = 1
    mat = mat[:,:4]
    return mat

In [25]:
#获得Sequence的序号表达
d = {'a':0, 'A':0, 'g':1, 'G':1, 'c':2, 'C':2, 't':3, 'T':3, 'N':4, 'n':4}
def seq_to_digital(seq):
    digital_seq = np.zeros((len(seq),1))  
    for i in range(len(seq)):
        digital_seq[i] =  d[seq[i]]
    return digital_seq

In [23]:
PESeq=dict()
PESeq['Pro-index']=[]
PESeq['Pro-Seq']=[]
PESeq['Pro-Kmer']=[]
PESeq['Pro-Digi']=[]
PESeq['Ehr-index']=[]
PESeq['Ehr-Seq']=[]
PESeq['Ehr-Kmer']=[]
PESeq['Ehr_Digi']=[]
PESeq['label'] =[]

In [101]:
cut_length =5000
cut_length_haf = cut_length//2
for index in range( len( csvdata[csvkeys[0]])):
    pro_index = [csvdata['promoter_chrom'][index],csvdata['promoter_start'][index],csvdata['promoter_end'][index]]   
    ehr_index = [csvdata['enhancer_chrom'][index],csvdata['enhancer_start'][index] ,csvdata['enhancer_end'][index]]
    pro_seq =  genome[csvdata['promoter_chrom'][index]][int(csvdata['promoter_start'][index]) :int(csvdata['promoter_end'][index])].upper()
    ehr_seq =  genome[csvdata['enhancer_chrom'][index]] [int(csvdata['enhancer_start'][index]) : int(csvdata['enhancer_end'][index])].upper()    

    if len(pro_seq)<50:
        print index,'\tpro_index',pro_index
        continue
    if len(ehr_seq)<50:
        print index,'\tehr_index',ehr_index
        continue
    if len(pro_seq)>cut_length:
        print 'pro too long',pro_index
        median = (len(pro_seq)+1)//2
        pro_seq = pro_seq[(median-cut_length_haf):(median+cut_length_haf)]
    if len(ehr_seq)>cut_length:
        print 'ehr too long',ehr_index
        median = (len(ehr_seq)+1)//2
        ehr_seq = ehr_seq[(median-cut_length_haf):(median+cut_length_haf)]
    PESeq['Pro-index'].append(pro_index )
    PESeq['Pro-Seq'].append(pro_seq)
    PESeq['Pro-Kmer'].append( seq_to_kspec(pro_seq))
    PESeq['Pro-Digi'].append(seq_to_digital(pro_seq))
    #PESeq['Pro-Mat'].append(seq_to_mat(pro_seq))
    PESeq['Ehr-index'].append(ehr_index)
    PESeq['Ehr-Seq'].append(ehr_seq)
    PESeq['Ehr-Kmer'].append(seq_to_kspec(ehr_seq))
    PESeq['Ehr_Digi'].append(seq_to_digital(ehr_seq))
    #PESeq['Ehr-Mat'].append(seq_to_mat(ehr_seq))
    PESeq['label'].append(csvdata['label'][index])

1 	ehr_index ['chr1', '24136556', '24136600']
pro too long ['chr1', '27018647', '27025208']
pro too long ['chr1', '27018647', '27025208']
pro too long ['chr1', '28831819', '28837986']
pro too long ['chr1', '28831819', '28837986']
pro too long ['chr1', '28831819', '28837986']
22 	ehr_index ['chr1', '100274800', '100274848']
37 	ehr_index ['chr1', '167687775', '167687800']
50 	ehr_index ['chr1', '183623600', '183623606']
57 	ehr_index ['chr1', '207592200', '207592223']
64 	ehr_index ['chr1', '223886400', '223886445']
pro too long ['chr1', '234741625', '234749400']
68 	ehr_index ['chr1', '234637359', '234637400']
pro too long ['chr1', '234741625', '234749400']
72 	ehr_index ['chr10', '21810994', '21811000']
73 	ehr_index ['chr10', '21810994', '21811000']
pro too long ['chr10', '98029122', '98034542']
pro too long ['chr10', '98029122', '98034542']
pro too long ['chr10', '98029122', '98034542']
pro too long ['chr10', '98029122', '98034542']
pro too long ['chr10', '98029122', '98034542']
pro

In [107]:
range(10)[1:3]

[1, 2]

In [108]:
print '-'*50
for item in PESeq.keys():
    print item, np.shape(PESeq[item])
print 'pos data' , sum(int( item ) for item in PESeq['label'])
print 'neg data',sum(int( 1) if item =='0' else int(0) for item in PESeq['label'])



--------------------------------------------------
Ehr-Kmer (93444, 4096, 1)
Pro-Digi (93445,)
Ehr-Seq (93445,)
Pro-index (93449, 3)
Pro-Kmer (93449, 4096, 1)
Pro-Seq (93449,)
Ehr-index (93445, 3)
Ehr_Digi (93443,)
label (93443,)
pos data 10597
neg data 82846


In [109]:
print 'lengthest Enhancer',len(max(PESeq['Ehr-Seq'],key= lambda x:len(x)))
print  'lengthest Promoter',len(max(PESeq['Pro-Seq'],key= lambda x:len(x)))

lengthest Enhancer 9466
lengthest Promoter 18594


In [87]:
X=[]
Y=[]
for index in range( len( PESeq['label'])):
    Y.append(PESeq['label'][index])
    X.append(PESeq['Pro-Seq'][index]+PESeq['Ehr-Seq'][index])
print np.shape(X)
print np.shape(Y)
print X[1]
X = np.array(X)
Y  = np.array(Y)

(44236,)
(44236,)
TTTGCCATTTCAAAGAATCTTGGATTTTTCTCTGGGCTCCAGAGACACCCGTGCCTGACACCCCCAATTTGAACCCCCAGCGGATCATTGGGTTCCATTGCTGATGGGGACCTTGTGTTTGAGCTTGACTTTCCCTCCATCCATGCTGGAGATAAACGGAAGTCTTCGCGCAGGGGGACGATTGCAGGACGGGGCCGCTAGGGAGGAGGGCTCCCACACGTGGGGAATGGGGGTGGAGGAACCGCTGAAGCTCGTCTGCCCTTTGGAAAATCTCGTTGGTGTCCTAGCCCTGGGATCATTGTACTTACCAGGTTGGGGGGCTCTGCACCCAGAACTCCTCCCAGCCTCTGTCTGCCCGTCCAAGGTCCGGCGTGACTTCCGGTGGCTGATTTCTCATCTTGGGGAAATCCCCGCCCTGGGGGGGTCCGGTGTGGCCTGCAGGGAGCCTGACTCACCCGTCGGCTGAGTCAGGGGGTCCCAGCTGCCCTCGGATGTCTGCTGCACCGCTCCCCTCCCTCAGGCGGGGGACGGCAGGAAGGAGGGAGAGATGGGAGGGCGTGGGGGCGGGGCCGGGGGCGGTCTCCAGCCCCCGGCCCGGGCTCCGCCCTCTCCCGGGATCTGTGAAAGCACAGCCAGGGTGCGGGCGGGTGAGTGCCCACCTGGTGCAGGGTGTGAAGAGCGAGAGCGCGTGTATGAGCACGAGCGTCTGTGGTCTCTGTGTGTCCGTTTACATGATTGCGCCTGTGTCTGTGTGTGCGCATCTGCATGCGTGTCTTTTTGAGTGAATGTCTGTGTGTGACTTCTAGGGACGGCCAGGGAAGAGGAGTGTCCCCTCCCACCCTGGCCCCCCGTCTTCTCATCCTGGACTCCCTCTCTTCTCTCTCCTCCTTGGGGGGTGCTCTGGGCATCCGGGACCCCTGGCAAGGCTGCCGGGTGCTGGGAGGACGCAGAGGCCGCCGAGGGCGGAGGGAAGCGTCAGGCC

In [91]:
print '最长',len(max(X,key=lambda x:len(x)))

最长 20550


In [32]:
from keras.utils.np_utils import to_categorical

In [33]:

VALIDATION_SPLIT =0.2

np.random.seed(1234)
index = range(len(PESeq['label']))
np.random.shuffle(index)
nb_validation_samples = int(VALIDATION_SPLIT*len(Y))

In [34]:

x_train = X[index[: - nb_validation_samples]]
y_train = Y[index[:-nb_validation_samples]]
x_val = X[index[-nb_validation_samples:]]
y_val =Y[index[-nb_validation_samples:]]



In [40]:
y_train = to_categorical(y_train) #one-hot 编码labels
y_val = to_categorical(np.asarray(y_val))

In [41]:
print 'x_train_shape:',np.shape(x_train)
print 'y_train_shape:',np.shape(y_train)

x_train_shape: (35389,)
y_train_shape: (35389, 2)


In [46]:
#使验证集正负样本数量相等
print np.shape(y_val)

print 'valid neg-pos num:' ,sum(y_val)


count = defaultdict()
count['0']=[]
count['1']=[]
for i in range(8847):
    if y_val[i][0]==1:
        count['0'].append(i)
    else:
        count['1'].append(i)
print len(count['0']),len(count['1'])


index_my = count['0'][:len(count['1'])]+count['1']

x_val_my = x_val[index_my]
y_val_my  =y_val[index_my,:]

print  'x_val_shape:',np.shape(x_val_my)
print  'y_val_shape:',np.shape(y_val_my)

(8847, 2)
valid neg-pos num: [ 8446.   401.]
8446 401
x_val_shape: (802,)
y_val_shape: (802, 2)


In [20]:
import pickle as pkl

In [25]:
import hickle as hkl

In [28]:
output = open('./Gen_data/10_PESeq.hkl','w')

hkl.dump(PESeq,output)

In [23]:
inputhkl = open('./Gen_data/10_PESeq.hkl','r')

PESeq1 =hkl.load(inputhkl)

In [ ]:
print PESeq1.keys()

# Make Network

In [47]:
from keras.layers import Embedding
from keras.layers import Dense,Input,Activation
from keras.layers import Embedding, LSTM, Bidirectional,GRU,InputLayer
from keras.models import Model,Sequential
from  keras.regularizers import ActivityRegularizer
from keras.layers.core import Dropout

In [48]:
from keras import layers

In [71]:
EMBEDDING_DIM =5
MAX_SEQUENCE_LENGTH=500
word_index = 3 #总共四种 ATCG
embedding_matrix =np.array(np.double(np.eye(4,5)))
print embedding_matrix,embedding_matrix.shape

[[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]] (4, 5)


In [72]:
seq_int = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

In [73]:
l_embed = Embedding(word_index+1,
                    EMBEDDING_DIM,
                    weights = [embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable=False,dropout=0.)

In [74]:
embeded_seq = l_embed(seq_int)

In [75]:
l_lstm =Bidirectional(LSTM(2,return_sequences=True))(embeded_seq)

In [79]:
l_lstm_drop = Dropout(0.3)(l_lstm)
l_att = AttLayer()(l_lstm_drop)
l_att_drop = Dropout(0.5)(l_att)
preds = Dense(len( y_train[0]), activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_att)
model  = Model (seq_int,preds)

In [80]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention LSTM network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val_my, y_val_my), nb_epoch=2, batch_size=100)

model fitting - attention LSTM network
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 500)           0                                            
____________________________________________________________________________________________________
embedding_4 (Embedding)          (None, 500, 5)        20          input_3[0][0]                    
Total params: 20
Trainable params: 0
Non-trainable params: 20
____________________________________________________________________________________________________


ValueError: Error when checking model input: expected input_3 to have shape (None, 500) but got array with shape (35389, 1)

In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=10, batch_size=10)